# Python で気軽に化学・化学工学
# 第 9 章　モデルの適用範囲・アンサンブル学習
## 9.3 One-Class Support Vector Machine (OCSVM)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット(descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの変数 (記述子) で化学構造が数値化されています。記述子は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# トレーニングデータとテストデータとに分割。shuffle=False として、上から順に(沸点が低い傾向のある化合物のみ)トレーニングデータを選択し、残りをテストデータとします
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=244, shuffle=False)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

### OCSVM によるデータ密度の計算および AD の設定

OCSVM によって計算されるデータ密度で AD を設定します。f(x) の値が 0 以上であれば、AD 内と考えられます。

はじめに、グラム行列の分散の最大化により、ガウシアンカーネルにおける *γ* を最適化します。

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
nonlinear_ocsvm_gammas = 2 ** np.arange(-20, 11, dtype=float) # γの候補

In [ ]:
nonlinear_ocsvm_gammas # 念のため確認

In [ ]:
from scipy.spatial.distance import cdist # トレーニングデータにおけるユークリッド距離に使用

In [ ]:
square_of_euclidean_distance = cdist(autoscaled_x_train, autoscaled_x_train, metric='sqeuclidean')

In [ ]:
square_of_euclidean_distance # 念のため確認

In [ ]:
variance_of_gram_matrix = [] # この変数にグラム行列の分散を入れていきます

In [ ]:
for nonlinear_ocsvm_gamma in nonlinear_ocsvm_gammas:
    gram_matrix = np.exp(- nonlinear_ocsvm_gamma * square_of_euclidean_distance)
    variance_of_gram_matrix.append(gram_matrix.var(ddof=1))

In [ ]:
optimal_nonlinear_ocsvm_gamma = nonlinear_ocsvm_gammas[variance_of_gram_matrix.index(max(variance_of_gram_matrix))] # グラム行列の分散が最大となるγ

In [ ]:
optimal_nonlinear_ocsvm_gamma # 念のため確認

OCSVM モデルの構築

In [ ]:
from sklearn.svm import OneClassSVM # OCSVM の実行に使用

In [ ]:
nonlinear_ocsvm_nu = 0.045

In [ ]:
ad_model = OneClassSVM(kernel='rbf', gamma=optimal_nonlinear_ocsvm_gamma, nu=nonlinear_ocsvm_nu) # OCSVM モデルを表す変数の作成

In [ ]:
ad_model.fit(autoscaled_x_train) # モデル構築

#### テストデータに対して、AD の内か外かを判定

In [ ]:
output_of_ocsvm_model_test = ad_model.decision_function(autoscaled_x_test)

In [ ]:
output_of_ocsvm_model_test # 念のため確認

In [ ]:
inside_ad_flag_test = output_of_ocsvm_model_test >= 0 # AD 内であったら True

In [ ]:
inside_ad_flag_test # 念のため確認

In [ ]:
inside_ad_flag_test = pd.DataFrame(inside_ad_flag_test)

In [ ]:
inside_ad_flag_test.index = x_test.index
inside_ad_flag_test.columns = ['TRUE if samples are inside of AD']

In [ ]:
inside_ad_flag_test.to_csv('inside_ad_flag_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。